In [2]:
from hdf5libs import HDF5RawDataFile

import daqdataformats
import detdataformats
from daqdataformats import FragmentType
from rawdatautils.unpack.daphne import *

import matplotlib.pyplot as plt

In [3]:
filename  = "/data0/np04hd_raw_run025016_0000_dataflow4_datawriter_0_20240415T141456.hdf5.copied"
run       = filename.split('_')[2]
h5_file   = HDF5RawDataFile(filename)
records   = h5_file.get_all_record_ids()
nrecords  = len(records)
nskip     = 1
det       = 'HD_PDS'

HDF5-DIAG: Error detected in HDF5 (1.12.0) thread 0:
  #000: /tmp/root/spack-stage/spack-stage-hdf5-1.12.0-nwlxllzr7neqtcsd575mrf2jhie4sfbd/spack-src/src/H5F.c line 793 in H5Fopen(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #001: /tmp/root/spack-stage/spack-stage-hdf5-1.12.0-nwlxllzr7neqtcsd575mrf2jhie4sfbd/spack-src/src/H5VLcallback.c line 3500 in H5VL_file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: /tmp/root/spack-stage/spack-stage-hdf5-1.12.0-nwlxllzr7neqtcsd575mrf2jhie4sfbd/spack-src/src/H5VLcallback.c line 3465 in H5VL__file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: /tmp/root/spack-stage/spack-stage-hdf5-1.12.0-nwlxllzr7neqtcsd575mrf2jhie4sfbd/spack-src/src/H5VLnative_file.c line 100 in H5VL__native_file_open(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #004: /tmp/root/spack-stage/spack-stage-hdf5-1.12.0-nwl

RuntimeError: Issue when opening file /eospublic.cern.ch:1094//eos/experiment/neutplatform/protodune/dune/hd-protodune/c4/87/np04hd_raw_run025017_0000_dataflow4_datawriter_0_20240415T144028.hdf5: Unable to open file /eospublic.cern.ch:1094//eos/experiment/neutplatform/protodune/dune/hd-protodune/c4/87/np04hd_raw_run025017_0000_dataflow4_datawriter_0_20240415T144028.hdf5 (File accessibility) Unable to open file

In [22]:
### Reading HDF5 files and saving the important information in a vector
info_perfragment = []

for r in records[nskip:nrecords]:
    
    pds_geo_ids    = list(h5_file.get_geo_ids_for_subdetector(r,detdataformats.DetID.string_to_subdetector(det)))
    for gid in pds_geo_ids:

        frag = h5_file.get_frag(r,gid)
        frag_id    = str(frag).split(' ')[3][:-1]
        fragType = frag.get_header().fragment_type
        
        # For self trigger
        if fragType == FragmentType.kDAPHNE.value:    
            timestamps = np_array_timestamp(frag)
            adcs       = np_array_adc(frag)
            channels   = np_array_channels(frag)

        # For full_stream
        elif fragType == 13:
            timestamps = np_array_timestamp_stream(frag)
            adcs       = np_array_adc_stream(frag)
            channels   = np_array_channels_stream(frag)[0]

        info_perfragment.append([frag_id, gid, channels, adcs, timestamps])
        

In [23]:
#### Persistence of each channel per geo_id

gid = list(set(item[1] for item in info_perfragment))

for g in gid:
    ch_ref = None
    wf_list = None
    match = 'no'

    for index, item in enumerate(info_perfragment):
        fragment, geo_id, channels, waveforms, timestamps = item
        
        if geo_id == g:
            if match == 'no':
                ch_ref = list(set(channels))
                wf_list = [[] for _ in range(len(channels))]
                match = 'yes'

            for i, wf in enumerate(waveforms):
                ch_index = ch_ref.index(channels[i])
                wf_list[ch_index].append(wf)

        if geo_id != g or index == (len(info_perfragment) - 1):
            match = 'no'
            if ch_ref is not None:
                for c, ch in enumerate(ch_ref):
                    for wf in wf_list[c]:
                        plt.plot(wf)
                    plt.title(f'ID: {g} Channel = {ch}')
                    plt.show()

In [ ]:
baseline_low_limit = 0
baseline_upper_limit = 50

integration_low_limit = 150
integration_upper_limit = 180

In [ ]:
#### Charge histogram per channel
from scipy.signal import savgol_filter

gid = list(set(item[1] for item in info_perfragment))

for g in gid:
    ch_ref = None
    wf_list = None
    match = 'no'

    for index, item in enumerate(info_perfragment):
        fragment, geo_id, channels, waveforms, timestamps = item
        
        if geo_id == g:
            if match == 'no':
                charge_list = [[] for _ in range(len(channels))]
                ch_ref = list(set(channels))
                match = 'yes'

            for i, wf in enumerate(waveforms):
                baseline = (savgol_filter(wf[baseline_low_limit: baseline_upper_limit], 2, 1)).mean()
                charge = sum(baseline - savgol_filter(wf[integration_low_limit: integration_upper_limit], 2, 1))
                
                ch_index = ch_ref.index(channels[i])             
                charge_list[ch_index].append(charge)
                             

        if geo_id != g or index == (len(info_perfragment) - 1):
            match = 'no'
            if ch_ref is not None:
                for c, ch in enumerate(ch_ref):
                    plt.hist(charge_list[c], bins = int(len(charge_list[c])/4))
                    plt.title(f'ID: {g} Channel = {ch}')
                    plt.show()